In [1]:
import pandas as pd
from llm import Granite
import json

# Data Preview

- There is missing value in COD, Credit Card, Debit Card, and EFT column. 
- We dont need to handle the missing value. Due to Its normal because buyer must pick one of payment method to order.

In [2]:
all_order = pd.read_excel("data/all_order.xlsx")
all_order['Order Date'] = pd.to_datetime(all_order['Order Date'])
all_order.head()

,Order ID,Order Date,State,Name,Region,Sale Price,Cost per Item,COD,Credit Card,Debit Card,EFT,Quantity,Product Category,Product Name
0,B-25757,2023-08-21,Florida,Reynolds,Southeast,106,13.00,7.0,NaN,NaN,NaN,7,Clothing,Hat
1,B-25757,2023-08-21,Florida,Reynolds,Southeast,14,9.00,NaN,1.0,NaN,NaN,1,Clothing,Hat
2,B-25757,2023-08-21,Florida,Reynolds,Southeast,165,45.00,NaN,3.0,NaN,NaN,3,Clothing,Scarf
3,B-25757,2023-08-21,Florida,Reynolds,Southeast,17,3.33,NaN,NaN,NaN,3.0,3,Clothing,Hat
4,B-25757,2023-08-21,Florida,Reynolds,Southeast,17,7.50,NaN,NaN,4.0,NaN,4,Clothing,Skirt


In [3]:
all_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1587 entries, 0 to 1586
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Order ID          1587 non-null   object        
 1   Order Date        1587 non-null   datetime64[ns]
 2   State             1587 non-null   object        
 3   Name              1587 non-null   object        
 4   Region            1587 non-null   object        
 5   Sale Price        1587 non-null   int64         
 6   Cost per Item     1587 non-null   float64       
 7   COD               175 non-null    float64       
 8   Credit Card       729 non-null    float64       
 9   Debit Card        213 non-null    float64       
 10  EFT               470 non-null    float64       
 11  Quantity          1587 non-null   int64         
 12  Product Category  1587 non-null   object        
 13  Product Name      1587 non-null   object        
dtypes: datetime64[ns](1), fl

# Add RFM (Recency, Frequency, Monetary) Features

- Add RFM features
- Sample 20% of customers, following the Pareto principle and to optimize costs when passing data to LLM models

In [3]:
reference_date = all_order['Order Date'].max() + pd.Timedelta(days=1)
rfm = all_order.groupby('Name').agg({
    'Order Date': lambda x: (reference_date - x.max()).days,  
    'Order ID': 'nunique',                                    
    'Sale Price': 'sum'                                       
}).reset_index()
rfm.columns = ['Name', 'Recency', 'Frequency', 'Monetary']
rfm

,Name,Recency,Frequency,Monetary
0,Adams,256,3,5449
1,Aguilar,173,1,1688
2,Alexander,21,1,78
3,Allen,66,3,2223
4,Alvarado,116,1,38
...,...,...,...,...
331,Wong,93,1,3287
332,Wood,250,2,669
333,Woods,38,1,108
334,Wright,254,3,587


In [4]:
rfm_sampled = rfm.sample(frac=0.2, random_state=42)
rfm_sampled

,Name,Recency,Frequency,Monetary
72,Diaz,235,2,1201
110,Gray,286,2,233
298,Tran,92,1,188
108,Graham,184,1,74
277,Silva,332,1,291
...,...,...,...,...
193,Miller,26,4,4901
299,Tucker,218,1,418
295,Thomas,47,3,1777
328,Williamson,327,1,87


# Analysis using IBM Granite

- Classify customer segment based on RFM
- Identify customer profile based on segment
- Determine buyer persona based on segment
- Plan marketing strategies based on buyer persona

*note*: All task is done by IBM Granite

In [5]:
granite = Granite(all_order)

### Classify customer segment

In [6]:
customer_categories = granite.classify_customer_segment(rfm_sampled)
customer_categories

{'Loyal Customers': ['Douglas', 'Duncan'],
 'Big Spender': ['Arnold',
  'Vargas',
  'Perkins',
  'Sanders',
  'Sanchez',
  'Lambert'],
 'New Customers': ['Tran',
  'Gray',
  'Soto',
  'Vargas',
  'Herrera',
  'Ruiz',
  'Warren',
  'Mason',
  'Wheeler'],
 'At Risk': ['Carpenter',
  'Bishop',
  'Frazier',
  'Mills',
  'Keller',
  'Jimenez',
  'Contreras',
  'Castillo',
  'Bryant',
  'Martinez',
  'Patterson',
  'Hall',
  'Schultz',
  'Vasquez',
  'Dunn',
  'Marshall',
  'Carter',
  'Gutierrez',
  'Cruz',
  'Bryant',
  'Martin',
  'Wheeler',
  'Coleman',
  'Evans',
  'Miller',
  'Tucker',
  'Thomas',
  'Williamson'],
 'Churned': []}

In [ ]:
def assign_label(name):
    labels = [label for label, names in customer_categories.items() if name in names]
    return ', '.join(labels) if labels else 'Not Segmented'

# Apply function
all_order['Customer Segment'] = all_order['Name'].apply(assign_label)

| Customer Segment | Description |
|-----------------|-------------|
| Loyal Customers | Repeat buyers making regular purchases |
| Big Spenders | High-value customers with less frequent but expensive purchases (average ~$500). |
| New Customers | Entry-level buyers with low basket size (~$47). |
| At Risk | Previously engaged customers who now show signs of drop-off, purchasing cheaper items in bulk but with low average value (~$35) |
| Churned | High-value customers who have stopped purchasing altogether. |


### Identify customer profile

In [ ]:
at_risk = all_order[all_order['Customer Segment']=='At Risk']
customer_at_risk = granite.get_customer_profile(at_risk)
customer_at_risk

{'Majority Region': 'West',
 'Payment Method': {'Most': 'Credit Card', 'Least': 'EFT'},
 'Product Category': {'Most': 'Clothing', 'Least': 'Electronics'},
 'Product Name': {'Most': 'Hat', 'Least': 'Accessories'},
 'Order Date': {'Most': '2023-10-01', 'Least': "I don't know"},
 'Quantity': {'Most': 7, 'Least': 3},
 'Cost per Item': {'Most': 4.71, 'Least': 9.0},
 'Sale Price': {'Most': 45, 'Least': 21}}

In [ ]:
big_spender = all_order[all_order['Customer Segment']=='Big Spender']
customer_big_spender = granite.get_customer_profile(big_spender)
customer_big_spender

{'Majority Region': 'Northeast',
 'Payment Method': {'Most': 'Credit Card', 'Least': 'EFT'},
 'Product Category': {'Most': 'Electronics', 'Least': 'Furniture'},
 'Product Name': {'Most': 'Phones', 'Least': 'Bookcases'},
 'Order Date': {'Most': '2023-05-20', 'Least': "I don't know"},
 'Quantity': {'Most': '4.0', 'Least': '1.0'},
 'Cost per Item': {'Most': '161.0', 'Least': '131.0'},
 'Sale Price': {'Most': '785.0', 'Least': '258.0'}}

In [ ]:
churned = all_order[all_order['Customer Segment']=='Churned']
customer_churned = granite.get_customer_profile(churned)
customer_churned 

{'Majority Region': 'Northeast',
 'Payment Method': {'Most': 'Credit Card', 'Least': 'NaN'},
 'Product Category': {'Most': 'Furniture', 'Least': 'NaN'},
 'Product Name': {'Most': 'Bookcases', 'Least': 'NaN'},
 'Order Date': {'Most': '2023-01-25', 'Least': 'NaN'},
 'Quantity': {'Most': '2', 'Least': 'NaN'},
 'Cost per Item': {'Most': '211.0', 'Least': 'NaN'},
 'Sale Price': {'Most': '662', 'Least': 'NaN'}}

In [ ]:
loyal_customer = all_order[all_order['Customer Segment']=='Loyal Customers']
customer_loyal_customer = granite.get_customer_profile(loyal_customer)
customer_loyal_customer

{'Majority Region': 'Southeast',
 'Payment Method': {'Most': 'Credit Card', 'Least': 'EFT'},
 'Product Category': {'Most': 'Clothing', 'Least': 'Furniture'},
 'Product Name': {'Most': 'Hat', 'Least': 'Bookcases'},
 'Order Date': {'Most': '2023-02-01', 'Least': '2023-05-23, 2023-07-12'},
 'Quantity': {'Most': '7', 'Least': '8'},
 'Cost per Item': {'Most': 'NaN', 'Least': '3.0'},
 'Sale Price': {'Most': '338', 'Least': '10'}}

In [ ]:
new_customer = all_order[all_order['Customer Segment']=='New Customers']
customer_new_customer = granite.get_customer_profile(new_customer)
customer_new_customer

{'Majority Region': 'West',
 'Payment Method': {'Most': 'EFT', 'Least': 'COD'},
 'Product Category': {'Most': 'Clothing', 'Least': 'N/A'},
 'Product Name': {'Most': 'Shirt', 'Least': 'N/A'},
 'Order Date': {'Most': '2023-09-09', 'Least': 'N/A'},
 'Quantity': {'Most': '2', 'Least': 'N/A'},
 'Cost per Item': {'Most': '33.5', 'Least': 'N/A'},
 'Sale Price': {'Most': '47', 'Least': 'N/A'}}

### Saving customer profile to JSON (Ignore)

In [ ]:
json_string = json.dumps(customer_new_customer, indent=4)  

with open("customer_profile/new_customer.json", "w") as file:
    json.dump(customer_new_customer, file, indent=4)

In [ ]:
json_string = json.dumps(customer_at_risk, indent=4) 

with open("customer_profile/at_risk_customer.json", "w") as file:
    json.dump(customer_at_risk, file, indent=4)

In [ ]:
json_string = json.dumps(customer_loyal_customer, indent=4) 

with open("customer_profile/loyal_customer.json", "w") as file:
    json.dump(customer_loyal_customer, file, indent=4)

In [ ]:
json_string = json.dumps(customer_churned, indent=4)  

with open("customer_profile/churned_customer.json", "w") as file:
    json.dump(customer_churned, file, indent=4)

In [ ]:
json_string = json.dumps(customer_big_spender, indent=4)  

with open("customer_profile/big_spender.json", "w") as file:
    json.dump(customer_big_spender, file, indent=4)

### Customer profile summary

In [4]:
import json
with open('customer_profile/new_customer.json') as f:
    new_customer_data = json.load(f)

with open('customer_profile/loyal_customer.json') as f:
    loyal_customer_data = json.load(f)

with open('customer_profile/churned_customer.json') as f:
    churned_customer_data = json.load(f)

with open('customer_profile/big_spender_customer.json') as f:
    big_spender_customer_data = json.load(f)

with open('customer_profile/at_risk_customer.json') as f:
    at_risk_customer_data = json.load(f)


json_data = {
    'new_customer': new_customer_data,
    'loyal_customer': loyal_customer_data,
    'churned_customer': churned_customer_data,
    'big_spender_customer': big_spender_customer_data,
    'at_risk_customer': at_risk_customer_data,
}


In [6]:
customer_profile_summary = granite.summarize_customer_profile(json_data)
customer_profile_summary

{'Loyal Customers': 'Southeast region, prefer Credit Card payment, most purchases are Clothing, Hats are most bought, average order date is 2023-02-01, average quantity is 7, cost per item varies, average sale price is 338.',
 'Big Spender': 'Northeast region, prefer Credit Card payment, Electronics are most purchased, Phones are most bought, average order date is 2023-05-20, average quantity is 4, cost per item ranges from 131 to 161, average sale price ranges from 258 to 785.',
 'New Customers': 'West region, prefer EFT payment, Clothing is most purchased, Shirts are most bought, average order date is 2023-09-09, average quantity is 2, cost per item is 33.5, average sale price is 47.',
 'At Risk': 'West region, prefer Credit Card payment, Clothing is most purchased, Hats are most bought, average order date is 2023-10-01, average quantity is 7, cost per item ranges from 4.71 to 9, average sale price ranges from 21 to 45.',
 'Churned': 'Northeast region, prefer Credit Card payment, Fur

| Customer Segment | Region | Payment Preference | Most Purchased Category | Most Bought Item | Avg Order Date | Avg Quantity | Cost per Item | Avg Sale Price |
|-----------------|--------|-------------------|----------------------|-----------------|---------------|--------------|--------------|---------------|
| Loyal Customers | Southeast | Credit Card | Clothing | Hats | 2023-02-01 | 7 | Varies | 338 |
| Big Spender | Northeast | Credit Card | Electronics | Phones | 2023-05-20 | 4 | 131-161 | 258-785 |
| New Customers | West | EFT | Clothing | Shirts | 2023-09-09 | 2 | 33.5 | 47 |
| At Risk | West | Credit Card | Clothing | Hats | 2023-10-01 | 7 | 4.71-9 | 21-45 |
| Churned | Northeast | Credit Card | Furniture | Bookcases | 2023-01-25 | 2 | 211 | 662 |


### Determine customer persona

In [ ]:
with open('customer_profile/summary.json') as f:
    summary_customer_data = json.load(f)

buyer_persona = granite.get_customer_persona(summary_customer_data)

with open("buyer_persona/buyer_persona.json", "w") as file:
    json.dump(buyer_persona, file, indent=4)

buyer_persona

{'Loyal Customers': {'persona_type': 'Repeat Buyer',
  'key_traits': 'Southeast region, Credit Card payment, Consistent Clothing purchases, Hats are most bought, avg 7 items, moderate spending',
  'average_purchase': '$338',
  'buyer_characteristic': 'Brand-attached, Quality-focused'},
 'Big Spender': {'persona_type': 'High-Value Customer',
  'key_traits': 'Northeast region, Credit Card payment, Electronics purchases, Phones are most bought, avg 4 items, high spending',
  'average_purchase': '$500',
  'buyer_characteristic': 'Brand-attached, Quality-focused, Less price-sensitive'},
 'New Customers': {'persona_type': 'Price-Sensitive Buyer',
  'key_traits': 'West region, EFT payment, Clothing purchases, Shirts are most bought, avg 2 items, low spending',
  'average_purchase': '$47',
  'buyer_characteristic': 'Price-sensitive, Value-focused'},
 'At Risk': {'persona_type': 'Declining Value Customer',
  'key_traits': 'West region, Credit Card payment, Clothing purchases, Hats are most boug

| Customer Segment | Persona Type | Key Traits | Average Purchase | Buyer Characteristic |
|-----------------|--------------|------------|------------------|---------------------|
| Loyal Customers | Repeat Buyer | Southeast region, Credit Card payment, Consistent Clothing purchases, Hats are most bought, avg 7 items, moderate spending | $338 | Brand-attached, Quality-focused |
| Big Spender | High-Value Customer | Northeast region, Credit Card payment, Electronics purchases, Phones are most bought, avg 4 items, high spending | $500 | Brand-attached, Quality-focused, Less price-sensitive |
| New Customers | Price-Sensitive Buyer | West region, EFT payment, Clothing purchases, Shirts are most bought, avg 2 items, low spending | $47 | Price-sensitive, Value-focused |
| At Risk | Declining Value Customer | West region, Credit Card payment, Clothing purchases, Hats are most bought, avg 7 items, low spending | $35 | Price-sensitive, Less brand-attached |
| Churned | Lost High-Value Customer | Northeast region, Credit Card payment, Furniture purchases, Bookcases are most bought, avg 2 items, high spending | $662 | Brand-attached, Quality-focused |


### Plan marketing strategies

In [ ]:
with open('customer_persona.json') as f:
    customer_persona_data = json.load(f)

customer_persona_recommendation = granite.get_recommendation(customer_persona_data)
customer_persona_recommendation

{'Loyal Customers': 'Offer exclusive discounts on hats, loyalty program rewards, and new clothing line notifications',
 'Big Spender': 'Promote new phone releases, exclusive access to sales, and premium customer service',
 'New Customers': 'Provide introductory discounts, bundle deals on shirts, and free shipping offers',
 'At Risk': 'Implement win-back campaigns with targeted discounts on hats, free shipping, and promotional bundles',
 'Churned': 'Offer personalized re-engagement campaigns, high-value discounts on bookcases, and exclusive access to new furniture lines'}

| Customer Segment | Recommended Marketing Strategies |
|-----------------|----------------------------------|
| Loyal Customers | - Offer exclusive discounts on hats<br>- Loyalty program rewards<br>- New clothing line notifications |
| Big Spender | - Promote new phone releases<br>- Exclusive access to sales<br>- Premium customer service |
| New Customers | - Provide introductory discounts<br>- Bundle deals on shirts<br>- Free shipping offers |
| At Risk | - Implement win-back campaigns with targeted discounts on hats<br>- Free shipping<br>- Promotional bundles |
| Churned | - Offer personalized re-engagement campaigns<br>- High-value discounts on bookcases<br>- Exclusive access to new furniture lines |
